<a href="https://colab.research.google.com/github/DelmiroDaladier/NLP-studies/blob/master/text_summarizarion/text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import re
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords


from keras import backend as K
from keras.models import load_model
from tensorflow.keras.models import Model
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional, Attention

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
dataset = pd.read_csv('Reviews.csv', encoding="utf-8", nrows=10000)

In [6]:
dataset.sample(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
9430,9431,B006N3IG4K,A154WMWOARJHLI,Gary Rosenfeld,2,2,5,1284681600,Great Coffee,This is an excellent blend perfect for any tim...
6469,6470,B000FNEX50,A12SAU7NR89O98,A. Olson,0,0,1,1350950400,Just Terrible.,We have a gluten free household of 5. No one ...
6358,6359,B001AW9PTO,A33LQ9MA6PCUBK,W. Chiu,0,1,2,1309219200,too salty,too salty -- enough said. I order the wood smo...
1590,1591,B001CWZXIY,A2TPNALNIKN96P,Vanessa,0,0,5,1329091200,My dog loves this dog food.,"My dog loves, loves this dog food.I had to sa..."
7775,7776,B000G7VYWU,ASRUYRC3R5J6Q,Betsy Heimbuch,1,1,3,1309046400,"IF YOU HAVE A PEANUT ALLERGY, BEWARE",My husband says these pretzels are great and h...


In [7]:
dataset.drop_duplicates(subset ='Text', inplace=True)
dataset.dropna(inplace=True)

In [8]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}



In [9]:
stop_words = set(stopwords.words('english'))
def clean_text(text):
    text = str(text)
    formatted_text = text.lower()
    formatted_text = re.sub(r'\([^)]*\)', '', formatted_text)
    formatted_text = re.sub('"','', formatted_text)
    formatted_text = re.sub(r'<.*?>', '', formatted_text)
    formatted_Text = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in formatted_text.split(" ")])    
    formatted_text = re.sub(r"'s\b","", formatted_text)
    formatted_text = re.sub("[^a-zA-Z]", " ", formatted_text) 
    tokens = [word for word in formatted_text.split() if word not in stop_words]
    

    return (" ".join(tokens).strip())

In [10]:
dataset['cleaned_texts'] = dataset['Text'].apply(lambda x: clean_text(x))
dataset['cleaned_summaries'] = dataset['Summary'].apply(lambda x: clean_text(x))

In [11]:
dataset['cleaned_texts'].replace('', np.nan, inplace=True)
dataset['cleaned_texts'].dropna(axis=0, inplace=True)
dataset['cleaned_summaries'].replace('', np.nan, inplace=True)
dataset['cleaned_summaries'].dropna(axis=0, inplace=True)

In [12]:
dataset['cleaned_texts'] = dataset['cleaned_texts'].apply(lambda x : '<SOS> ' + str(x) + ' <EOS>')
dataset['cleaned_summaries'] = dataset['cleaned_summaries'].apply(lambda x : '<SOS> ' + str(x) + ' <EOS>')

In [13]:
text_word_to_index = {}
text_index_to_word = {}

index = 0
for text in dataset['cleaned_texts']:
  for word in text.split():
    if word not in text_word_to_index.keys():
      text_word_to_index[word] = index
      text_index_to_word[index] = word
      index += 1

In [14]:
text_vocab_size = len(text_word_to_index.keys())

In [15]:
summary_word_to_index = {}
summary_index_to_word = {}

index = 0
for text in dataset['cleaned_summaries']:
  for word in text.split():
    if word not in summary_word_to_index.keys():
      summary_word_to_index[word] = index
      summary_index_to_word[index] = word
      index += 1

In [16]:
summary_vocab_size = len(summary_word_to_index.keys())

In [17]:
max_text_lenght = max([len(text.split()) for text in dataset['cleaned_texts']])
max_summary_lenght = max([len(summary.split()) for summary in dataset['cleaned_summaries']]) 

In [18]:
X_train, X_test, y_train, y_test = train_test_split(dataset['cleaned_texts'], dataset['cleaned_summaries'], test_size=0.1)

In [19]:
def batch_data_generator(x=X_train, y=y_train, batch_size=128):

  while True:
    for j in range(0, len(x), batch_size):
      encoder_input_data = np.zeros((batch_size, max_text_lenght), dtype="float32")
      decoder_input_data = np.zeros((batch_size, max_summary_lenght), dtype="float32")
      decoder_target_data = np.zeros((batch_size, max_summary_lenght, summary_vocab_size), dtype="float32")

      for i, (input_text, target_summary) in enumerate(zip(x[j:j+batch_size], y[j:j+batch_size])):
        for t, word in enumerate(input_text.split()):
          encoder_input_data[i, t] = text_word_to_index[word]
        for t, word in enumerate(target_summary.split()):
          if t < len(target_summary.split())-1:
            decoder_input_data[i, t] = summary_word_to_index[word]

          if t > 0:
            decoder_target_data[i, t - 1, summary_word_to_index[word]] = 1.0

      yield([encoder_input_data, decoder_input_data], decoder_target_data) 

In [42]:
K.clear_session()
latent_dim = 128

encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(text_vocab_size, latent_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(summary_vocab_size, latent_dim, mask_zero=True)
dec_emb = decoder_embedding(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(
    dec_emb,
    initial_state=encoder_states)
decoder_dense = Dense(summary_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [43]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 128)    2310656     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 128)    557952      input_2[0][0]                    
_______________________________________________________________________________________

In [44]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [ ]:
train_samples = len(X_train) # Total Training samples
val_samples = len(X_test)    # Total validation or test samples
batch_size = 128
epochs = 100

model.fit_generator(generator = batch_data_generator(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = batch_data_generator(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

model.save('text_summarization')

Epoch 1/100
66/66 [==============================] - 300s 5s/step - loss: 0.9422 - val_loss: 0.9111
Epoch 2/100
66/66 [==============================] - 284s 4s/step - loss: 0.8485 - val_loss: 0.9244
Epoch 3/100
66/66 [==============================] - 293s 4s/step - loss: 0.8363 - val_loss: 0.9369
Epoch 4/100
66/66 [==============================] - 279s 4s/step - loss: 0.8301 - val_loss: 0.9411
Epoch 5/100
66/66 [==============================] - 290s 4s/step - loss: 0.8218 - val_loss: 0.9469
Epoch 6/100
66/66 [==============================] - 282s 4s/step - loss: 0.8169 - val_loss: 0.9480
Epoch 7/100
66/66 [==============================] - 291s 4s/step - loss: 0.8048 - val_loss: 0.9504
Epoch 8/100
66/66 [==============================] - 293s 4s/step - loss: 0.7947 - val_loss: 0.9560
Epoch 9/100
66/66 [==============================] - 285s 4s/step - loss: 0.7827 - val_loss: 0.9525
Epoch 10/100
66/66 [==============================] - 281s 4s/step - loss: 0.7715 - val_loss: 0.9574

In [27]:
model = load_model('text_summarization')

encoder_inputs = model.input[0]
encoder_outputs, state_h_enc, state_c_enc = model.layers[4].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]
decoder_state_h_input = Input(shape=(latent_dim), name='hidden_state_input')
decoder_state_c_input = Input(shape=(latent_dim), name='cell_state_input')
decoder_states_inputs = [decoder_state_h_input, decoder_state_c_input]

dec_emb_layer = model.layers[3]
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = model.layers[5]
decoder_outputs, decoder_state_h, decoder_state_c = decoder_lstm(
    dec_emb, initial_state=decoder_states_inputs
)
decoder_states = [decoder_state_h, decoder_state_c]
decoder_dense = model.layers[6]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

In [33]:
def decode_sequence(input_seq):

  states_values = encoder_model.predict(input_seq)

  target_seq = np.zeros((1,1))

  target_seq[0, 0] = summary_word_to_index['<SOS>']

  decoded_sequence = ''
  stop_condition = False
  while not stop_condition:

    output_tokens, h, c = decoder_model([target_seq] + states_values)
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_token = summary_index_to_word[sampled_token_index]

    decoded_sequence += ' ' + sampled_token 

    if sampled_token == '<EOS>' or len(decoded_sequence.split()) > max_summary_lenght:
            stop_condition = True
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = sampled_token_index
    states_value = [h, c]
  
  return decoded_sequence


In [41]:
X_test.reset_index(drop=True, inplace=True) 
y_test.reset_index(drop=True, inplace=True) 
test_gen = batch_data_generator(x=X_train, y=y_test, batch_size=1)
for seq_index in range(10):
  input_seq = next(test_gen)
  decoded_summary = decode_sequence(input_seq)
  print("-")
  print("Input sentence:", X_test[seq_index])
  print("Real sentence:", y_test[seq_index])
  print("Predicted sentence:", decoded_summary)


-
Input sentence: <SOS> really enjoy low calorie chips good flavor good crunch crisp wondering ordering case chips one could order favorites whole flavors say flavors desired great product stop manufacturing <EOS>
Real sentence: <SOS> yummy yummies <EOS>
Predicted sentence:  mill product <EOS>
-
Input sentence: <SOS> addicted gum tastes close real thing lose flavor compared apple pie strawberry best flavor series along mint chocolate chip <EOS>
Real sentence: <SOS> delicious <EOS>
Predicted sentence:  mill product <EOS>
-
Input sentence: <SOS> plugged direct windows controlled via ustream tv software interface worked well interim setup could get pro level interface one mic etc nice feels bit flimsy construction wise going take road <EOS>
Real sentence: <SOS> used audience mic pickup <EOS>
Predicted sentence:  mill coffee <EOS>
-
Input sentence: <SOS> brother owns keurig one cup coffee maker amazing device however drink coffee came across hot cocoa k cups took chance incredibly good giv